In [14]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum Enhanced Optimization for Radar and Communications Applications 


The Low Autocorrelation Binary Sequences (LABS) is an important and challenging optimization problem with applications related to radar, telecommunications, and other signal related applications. This CUDA-Q Academic module will focus on a clever quantum-enhanced hybrid method developed in a collaboration between Kipu Quantum, University of the Basque Country EHU, and NVIDIA for solving the LABS problem. (This notebook was jointly developed with input from all collaborators.)

Other CUDA-Q Academic modules like [Divide and Conquer MaxCut QAOA](https://github.com/NVIDIA/cuda-q-academic/tree/main/qaoa-for-max-cut) and [Quantum Finance](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb), demonstrate how quantum computing can be used outright to solve optimization problems. This notebook demonstrates a slightly different approach. Rather than considering QPUs as the tool to produce the final answer, it demonstrates how quantum can be used to enhance the effectiveness of leading classical methods.  

The benefits of such an approach were highlighted in [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1).  This notebook, co-created with the authors of the paper, will allow you to explore the findings of their research and write your own CUDA-Q code that builds a representative quantum-enhanced workflow for solving the LABS problem. Moreover, it will introduce advancements in counteradiabatic optimization techniques on which reduce the quantum resources required to run on a QPU.

**Prerequisites:** This lab assumes you have a basic knowledge of quantum computing, including operators, gates, etc.  For a refresher on some of these topics, explore the [Quick start to Quantum](https://github.com/NVIDIA/cuda-q-academic/tree/main/quick-start-to-quantum) series.

**In this lab you will:**
* 1. Understand the LABS problem and its relation ot radar and communication applications.
* 2. Solve LABS classically with memetic tabu search and learn about the limitations of such methods.
* 3. Code a couteradiabatic algorithm using CUDA-Q to produce approximate solutions to the LABS problem.
* 4. Use the CUDA-Q results to seed your tabu search and understand the potential benefits of this approach.


**Terminology you will use:**
* Low autocorrelation of binary sequences (LABS)
* counteradiabatic optimization
* memetic-tabu search

**CUDA-Q Syntax you will use:**
* cudaq.sample()
* @cudaq.kernel
* ry(), rx(), rz(), x(), h() 
* x.ctrl()

Run the code below to initialize the libraries you will need.

In [15]:
pip install cudaq


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: /home/jovyan/.qbraid/environments/zogs/pyenv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import cudaq
import numpy as np
from math import floor
import auxiliary_files.labs_utils as utils

## The LABS problem and applications

The **Low Autocorrelation Binary Sequences (LABS)** problem is fundamental to many applications, but originated with applications to radar. 

Consider a radar that monitors airport traffic.  The radar signal sent to detect incoming planes must have as much range as possible to ensure safe approaches are planned well in advance.  The range of a radar signal can be increased by sending a longer pulse.  However, in order to differentiate between multiple objects, pulses need to be short to provide high resolution. So, how do you handle situations where you need both?

One solution is a technique called pulse compression.  The idea is to send a long signal, but vary the phase at regular intervals such that the resolution is increased. Generally, the initial signal will encode a binary sequence of phase shifts, where each interval corresponds to a signal with a 0 or 180 degree phase shift. 

The tricky part is selecting an optimal encoding sequence.  When the signal returns, it is fed into a matched filter with the hope that a singular sharp peak will appear, indicating clear detection.  The autocorrelation of the original signal, or how similar the signal is to itself,  determines if a single peak or a messier signal with sidelobes will be detected. A signal should have high autocorrelation when overlayed on top of itself, but low autocorrelation when shifted with a lag. 

Consider the image below.  The signal on the left has a crisp single peak while the single on the right produces many undesirable sidelobes which can inhibit clear detection.  

<img src="images/quantum_enhanced_optimization_LABS/radar.png" width="800">


So, how do you select a good signal?   This is where LABS comes in, defining these questions as a binary optimization problem. Given a binary sequence of length $N$, $(s_1 \cdots s_N) \in {\pm 1}^N$, the goal is to minimize the following objective function.

$$ E(s) = \sum_{k=1}^{N-1} C_k^2 $$

Where $C_k$ is defined as. 

 $$C_k= \sum_{i=1}^{N-k} s_is_{i+k}$$


So, each $C_k$ computes how similar the original signal is to the shifted one for each offset value $k$.  To explore this more, try the interactive widget linked [here](https://nvidia.github.io/cuda-q-academic/interactive_widgets/labs_visualization.html).  See if you can select a very good and very poor sequence and see the difference for the case of $N=7$.

## Classical Solution of the LABS problem

The LABS problem is tricky to solve for a few reasons. First, the configuration space grows exponentially.  Second, underlying symmetries of the problem result in many degeneracies in the optimization landscape severely inhibiting local search methods. 

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 1:</h3>
    <p style="font-size: 16px; color: #333;">
Using the widget above, try to find some of the symmetries for the LABS problem. That is, for a fixed bitstring length, can you find patterns to produce the same energy with different pulse patterns. 
</div>

The best known performance for a classical optimization technique is Memetic Tabu search (MTS) which exhibits a scaling of $O(1.34^N)$.  The MTS algorithm is depicted below.  It begins with a randomly selected population of bitstrings and finds the best solution from them.  Then, a child is selected by sampling directly from or combining multiple bitstrings from the population.  The child is mutated with probability $p_{mutate}$ and then input to a tabu search, which performs a modified greedy local search starting from the child bitstring.  If the result is better than the best in the population, it is updated as the new leader and randomly replaces a  bitstring in the population.


<img src="images/quantum_enhanced_optimization_LABS/mts_algorithm.png" width="500">

Such an approach is fast, parallelizable, and allows for exploration with improved searching of the solution landscape.  

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 2:</h3>
    <p style="font-size: 16px; color: #333;">
Before exploring any quantum approach, get a sense for how MTS works by coding it yourself based generally on the figure above. Algorithms for the combine and mutate steps are provided below as used in the paper. You may need to research more specific details of the process, especially for how tabu search is performed. The MTS procedure should output and optimal bitstring and its energy.  Also, write a function to visualize the results including the energy distribution of the final population.
</div>



<img src="images/quantum_enhanced_optimization_LABS/combine_mutate.png" width="400">



In [17]:
#TODO - Write code to perform MTS

import cudaq
import numpy as np
from math import floor
import auxiliary_files.labs_utils as utils


import random 

#creates bitstrings of length N
def bitstring(N):
    string = []
    for bit in range(N):
        string.append(random.choice([0,1]))  
    return string



#creates s_max bitstrings of lenth N
def populate(s_max,N):
    pop = []
    for i in range(s_max):
        pop.append(bitstring(N))
    return pop




#makes a array from 0 to N
def cut(N):
    k_list = []
    for bit in range(N):
        k_list.append(bit)   
    return k_list




#cuts two parent bitstrings p1 and p2 of length N at index k and combines the halves into a new bitstring child of length N
def combine(p1,p2):
    if len(p1)==len(p2):
        k = random.choice(cut(len(p1)))
        child = p1[0:k]
        for element in p2[k:len(p2)]:
            child.append(element)
        return child



#pm probability of flippinng each bit in a bitstring s  
def mutate(s,pm):
    for i in range(len(s)):
        if random.random()<pm:
            if s[i]==0:
                s[i]=1
            if s[i]==1:
                s[i]=0
    return s



#performs Tabu search on an input bitstring population s0 and outputs best modified bitstring s
#zias code with my fixes
def tabu_search(s0):
    s = s0.copy()
    p = s0.copy()
    N = len(s0)
    Tabu_list = [0] * N
    M = random.randint(0,N) + (N // 2)

    temp_list = [] 
    E_list = []
    theta_min = int(M / 50)
    if (theta_min ==0):
        theta_min =1
    theta_max = int(M / 10)
    if (theta_max <theta_min):
        theta_max = theta_min 
    
    Es = labs_energy_list(s)
    for t in range(1,M+1):  #fixed this range
        temp_list.clear()
        E_list.clear()

        #list of indexes at time t
        for i in range(len(Tabu_list)): #range()
            if Tabu_list[i]<t:
                temp_list.append(i)
                #i can flip if tabu expiration is over 

        #exception: if everything is tabu, allow any move
        if len(Tabu_list)==0:
            temp_list = list(range(N)) #index 0,1,2..N is all fine to flip
        
        #evaluate energies (store in list too)
        #this is just your swap code
        for index in temp_list:
            cand = p.copy()
            cand[index] ^= 1  #xor gate, flips 0 and 1s
            E_list.append(labs_energy_list(cand))
        
        
         #choose i_star = index with smallest energy
        j_star = int(np.argmin(E_list))   # position in temp_list
        i_star = temp_list[j_star]        # actual bit index to flip
        Ep = E_list[j_star]            
        p[i_star] ^= 1 #xor swap

        #update tabu for bit
        Tabu_list[i_star] = t + random.randint(theta_min, theta_max)

        #update all
        if Ep < Es:
            s = p.copy()
            Es = Ep
    return s        



#need to calculate energy to compare
def labs_energy_list(bits): 
    s = np.array(bits, dtype=int) * 2 - 1  # 0->-1, 1->+1
    N = len(s)
    E = 0
    for k in range(1, N):
        ck = int(np.dot(s[:N-k], s[k:])) #dot product
        E += ck * ck
    return E


def MTS(popsize,stringsize,pc,target):
    pop = populate(popsize,stringsize) #creates a random list of strings 
    if random.random()<pc:
        child = combine(random.choice(pop),random.choice(pop)) 
        #creates a child string from two randomly selected parent strings with probability pc
    else:
        child = random.choice(pop)
        #If no child is created, an existing member of the population is randomly selected
    mutant = mutate(child,1/len(child))
    #mutates child
    s_best = tabu_search(mutant)
    #performs a tabu_search with the child as the starting string 
    while labs_energy_list(s_best)>target:
    #repeats child selection, mutation, and tabu search until target energy is reached
        if random.random()<pc:
            child = combine(random.choice(pop),random.choice(pop))
        else:
            child = random.choice(pop)
        mutant = mutate(child,1/len(child))
        s_best = tabu_search(mutant)
        random_index = random.randrange(len(pop))
        pop[random_index] = s_best
    return s_best #returns ideal string


In [18]:
popsize = 10
stringsize = 7
pc = 0.9
target = 3
print("Ideal String = ",*MTS(popsize,stringsize,pc,target),sep='')
print("E(best) = ", labs_energy_list(MTS(popsize,stringsize,pc,target)))

Ideal String = 1011000
E(best) =  3


## Building a Quantum Enhanced Workflow

Despite the effectiveness of MTS, it still exhibits exponential scaling  $O(1.34^N)$ behavior and becomes intractable for large $N$.  Quantum computing provides a potential alternative method for solving the LABS problem because the properties of entanglement, interference, and superpositon may allow for a better global search.  Recent demonstrations have even produced evidence that the quantum approximate optimization algorithm (QAOA) can be used to reduce the scaling of the LABS problem to $O(1.21^N)$ for $N$ between 28 and 40 with quantum minimum finding.

However, current quantum hardware limitations restrict solution to problems of greater than about $N=20$, meaning that it will be some time before quantum approaches can outperform the classical state of the art. It should also be noted that standard QAOA can struggle with LABS and require many layers to converge the parameters if other tricks are not employed.

The authors of [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1) cleverly explored an alternate path that combines quantum and classical approaches and might be able to provide a more near-term benefit.  Instead of expecting the quantum computer to solve the problem entirely, they asked how a quantum approach might enhance MTS.

The basic idea is that a quantum optimization routine could run first and the resulting state be sampled to produce a better population for MTS. Many such heuristics for defining the initial population are possible, but the rest of this notebook will explore their methodology, help you to build the workflow yourself, and allow you to analyze the benefits of their approach.

The first step of quantum enhanced MTS (QE-MTS) is to prepare a circuit with CUDA-Q that approximates the ground state of the Hamiltonian corresponding to the LABS problem. You could do this with any optimization algorithm such as QAOA or using an adiabatic approach.  (See the [Quantum Portfolio Optimization](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb) CUDA-Q Academic lab for a detailed comparison of these two common approaches.)

The authors of this work opted for an adiabatic approach (More on why later). Recall that the goal of an adiabatic optimization is to begin with a Hamiltonian that has an easily prepared ground state ($H_i$). Then, the adiabatic Hamiltonian $H_{ad}$ can be constructed as $H_{ad}(\lambda) = (1-\lambda)H_i +\lambda H_f $, where $\lambda$ is a function of time and $H_f$ is the Hamiltonian representing a qubit encoding of the LABS problem. 

$$H_f = 2 \sum_{i=1}^{N-2} \sigma_i^z \sum_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} \sigma_{i+k}^z 
+ 4 \sum_{i=1}^{N-3} \sigma_i^z \sum_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \sum_{k=t+1}^{N-i-t} \sigma_{i+t}^z \sigma_{i+k}^z \sigma_{i+k+t}^z$$

The authors also selected $H_i = \sum_i h^x_i \sigma^x_i $ which has an easily prepared ground state of $\ket{+}^{\otimes N}$.

The challenge for implementing the optimization procedure becomes selection of an operator that will quickly and accurately evolve to the ground state of $H_f$.  One approach is to use a so-called auxiliary countradiabatic (CD) term $H_{CD}$, which corrects diabatic transitions that jump out of the ground state during the evolution. The figure below demonstrates the benefit of using a CD correction.


<img src="images/quantum_enhanced_optimization_LABS/counteradiabatic.png" width="900">




An operator called the adiabatic gauge potential $A_{\lambda}$ is the ideal choice for the CD term as it suppresses all possible diabatic transitions, resulting in the following total system to evolve.

$$ H(\lambda) = H_{ad}(\lambda) + \lambda H_{CD} (\lambda) $$

$A(\lambda)$ is derrived from $H_{ad}(\lambda)$  (see paper for details) as it contains information about underlying physics of the problem. 

There is a problem though.  The $A(\lambda)$ term cannot be efficiently expressed exactly and needs to be approximated.  It also turns out that in the so called impulse regime, where the adiabatic evolution is very fast, $H_{cd} (\lambda)$ dominates $H_{ad}(\lambda)$, meaning that the final implementation corresponds to the operator $H(\lambda) = H^1_{cd}(\lambda)$ where  $H^1_{cd}(\lambda)$ is a first order approximation of $A(\lambda)$ (see equation 7 in the paper).

A final step is to use Trotterization to define the quantum circuit to apply $e^{-\theta (t) i H_{cd}}$. The details for this derivation are shown in the appendix of the paper. and result from equation B3 is shown below.  

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

It turns out that this implementation is more efficient than QAOA in terms of gate count. The authors calculated that for $N=67$, QAOA would require 1.4 million entangling gates while the CD approach derived here requires only 236 thousand entangling gates.


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 3:</h3>
    <p style="font-size: 16px; color: #333;">
At first glance, this equation might looks quite complicated. However, observe the structure and note two "blocks" of terms.  Can you spot them?  

They are 2 qubit terms that look like $R_{YZ}(\theta)$ or $R_{ZY}(\theta)$.

As well as 4 qubit terms that look like $R_{YZZZ}(\theta)$, $R_{ZYZZ}(\theta)$, $R_{ZZYZ}(\theta)$, or $R_{ZZZY}(\theta)$.

Thankfully the authors derive a pair of circuit implementations for the two and four qubit terms respectively, shown in the figures below.

Using CUDA-Q, write a kernel for each which will be used later to construct the full implementation.

* Hint: Remember that the adjoint of a rotation gate is the same as rotating in the opposite direction. 

* Hint: You may also want to define a CUDA-Q kernel for an R$_{ZZ}$ gate.

* Hint: Implementing a circuit from a paper is a great place where AI can help accelerate your work.  If you have access to a coding assistant, feel free to use it here.
</div>

<img src="images/quantum_enhanced_optimization_LABS/kernels.png" width="1300">


In [19]:
# the following code creates the circuit highlighted in excercise 3, which models the U(0,T) trotterizer function
# this function can be subdivided into two parts, into two qubit and four qubit rotations
# the two qubit block rotated around RYZ(theta) RZY(theta) 
# this requires 2 entangling gates (4 gates each) and 4 single qubit gates
# the second block is the quad-qubit rotational block, rotated around RYZZZ(theta),RZYZZ(Theta),RZZYZ(theta
# and RZZZY(theta). 
# the circuit breaks down the technique in quantum computing used to simulate the time evolution of a quantum system. 
# It breaks down complex, large-scale Hamiltonian simulations into a series of smaller, manageable steps

import cudaq
import numpy as np
import math as mat

theta = 0 #in degrees, change this var after finding in part 4
pie_2 = mat.pi / 2
pie = mat.pi

@cudaq.kernel
def dual_qubit_rotation():
    q = cudaq.qvector(2)
    rx(pie_2, q[1])
    h(q[1])
    h(q[0])
    x.ctrl(q[0], q[1])
    rz(theta, q[0])
    rz(theta, q[1])
    x.ctrl(q[1], q[0])
    rx(pie_2, q[0])
    rx(pie_2, q[1])
    rx(pie_2, q[0])

@cudaq.kernel
def quad_qubit_rotation():
    q = cudaq.qvector(4)
    ry(-pie_2, q[2])
    ry(pie_2, q[1])
    h(q[3])
    rx(-pie_2, q[0])
    h(q[2])
    h(q[1])
    h(q[0])
    x.ctrl(q[0], q[1])
    x.ctrl(q[2], q[3])
    rz(-pie_2, q[0])
    rz(-pie_2, q[1])
    rz(-pie_2, q[2])
    rz(-pie_2, q[3])
    x.ctrl(q[1], q[0])
    x.ctrl(q[2], q[3])
    rx(pie_2, q[0])
    rx(-pie_2, q[3])
    ry(pie_2, q[2])
    ry(-pie_2, q[1])
    rx(-pie_2, q[1])
    rx(pie_2, q[2])
    h(q[1])
    h(q[2])
    x.ctrl(q[1], q[2])
    rz(theta, q[1])
    rz(theta, q[2])
    x.ctrl(q[2], q[1])
    rx(pie_2, q[1])
    rx(pie, q[2])
    ry(pie_2, q[1])
    h(q[0])
    h(q[1])
    x.ctrl(q[0], q[1])
    rz(pie_2, q[0])
    rz(pie_2, q[1])
    x.ctrl(q[1], q[0])
    rx(pie_2, q[0])
    ry(-pie_2, q[1])
    h(q[1])
    h(q[2])
    x.ctrl(q[1], q[2])
    rz(theta, q[1])
    rz(theta, q[2])
    x.ctrl(q[2], q[1])
    rx(-pie, q[1])
    ry(pie_2, q[2])
    h(q[2])
    h(q[3])
    x.ctrl(q[2], q[3])
    rz(-pie_2, q[2])
    rz(-pie_2, q[3])
    x.ctrl(q[3], q[2])
    ry(-pie_2, q[2])
    rx(-pie_2, q[3])
    rx(-pie_2, q[2])
    h(q[1])
    h(q[2])
    x.ctrl(q[1], q[2])
    rz(theta, q[1])
    rz(theta, q[2])
    x.ctrl(q[2], q[1])
    rx(pie_2, q[1])
    rx(pie_2, q[2])
    ry(-pie_2, q[1])
    ry(pie_2, q[2])
    h(q[0])
    h(q[1])
    h(q[2])
    h(q[3])
    x.ctrl(q[0], q[1])
    x.ctrl(q[2], q[3])
    rz(pie_2, q[0])
    rz(pie_2, q[1])
    rz(pie_2, q[2])
    rz(pie_2, q[3])
    x.ctrl(q[1], q[0])
    x.ctrl(q[3], q[2])
    ry(pie_2, q[1])
    ry(-pie_2, q[2])
    rx(pie_2, q[3])

print(cudaq.draw(quad_qubit_rotation))


     ╭────────────╮╭───╮     ╭────────────╮╭───╮╭───────────╮     ╭───╮     »
q0 : ┤ rx(-1.571) ├┤ h ├──●──┤ rz(-1.571) ├┤ x ├┤ rx(1.571) ├─────┤ h ├─────»
     ├───────────┬╯├───┤╭─┴─╮├────────────┤╰─┬─╯├───────────┴╮╭───┴───┴────╮»
q1 : ┤ ry(1.571) ├─┤ h ├┤ x ├┤ rz(-1.571) ├──●──┤ ry(-1.571) ├┤ rx(-1.571) ├»
     ├───────────┴╮├───┤╰───╯├────────────┤     ├───────────┬╯├───────────┬╯»
q2 : ┤ ry(-1.571) ├┤ h ├──●──┤ rz(-1.571) ├──●──┤ ry(1.571) ├─┤ rx(1.571) ├─»
     ╰───┬───┬────╯╰───╯╭─┴─╮├────────────┤╭─┴─╮├───────────┴╮╰───┬───┬───╯ »
q3 : ────┤ h ├──────────┤ x ├┤ rz(-1.571) ├┤ x ├┤ rx(-1.571) ├────┤ h ├─────»
         ╰───╯          ╰───╯╰────────────╯╰───╯╰────────────╯    ╰───╯     »

################################################################################

                                                            ╭───────────╮╭───╮»
─────────────────────────────────────────────────────────●──┤ rz(1.571) ├┤ x ├»
╭───╮     ╭───────╮╭───╮╭───────────╮╭───────────╮╭───╮

There are a few additional items we need to consider before completing the final implementation of the entire circuit.  One simplification we can make is that for our problem the $h_i^x$ terms are all 1 and any $h_b^x$ terms are 0, and are only there for generalizations of this model. 

The remaining challenge is derivation of the angles that are used to apply each of the circuits you defined above. These are obtained from two terms $\lambda(t)$ and $\alpha(t)$.  


The $\lambda(t)$ defines an annealing schedule and is generally a Sin function which slowly "turns on" the problem Hamiltonian.  For computing our angles, we need the derivative of $\lambda(t)$.

The $\alpha$ term is a bit trickier and is the solution to a set of differential equations which minimize the distance between $H^1_{CD}(\lambda)$ and $A(\lambda)$.  The result is 

$$\alpha(t) = \frac{-\Gamma_1(t)}{\Gamma_2(t)} $$

Where $\Gamma_1(t)$ and $\Gamma_2(t)$ are defined in equations 16 and 17 of the paper and essentially depend on the structure of the optimization problem.  Curious learners can look at the functions in `labs_utils.py`  to see how these are computed, based on the problem size and specific time step in the Trotter process. 


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 4:</h3>
    <p style="font-size: 16px; color: #333;">
The `compute_theta` function, called in the following cells, requires all indices of the two and four body terms. These will be used again in our main kernel to apply the respective gates.  Use the products in the formula below to finish the function in the cell below.  Save them as `G2` and `G4` where each is a list of lists of indices defining the two and four term interactions. As you are translating an equation to a set of loops, this is a great opportunity to use an AI coding assistant.

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

</div>


In [20]:


def two_body_pairs(N):
    # two-body terms
    
    for i in range(1, N - 2):
        max_k = (N - i)//2
                 
        for k in range(1, max_k):
            j = i + k
            yield(i,j)


def four_body_pairs(N):
    # four-body terms

    for i in range(1, N - 3):
        max_t = (N - i - 1)//2

        for t in range(1, max_t):

            for k in range(t+1, N - i - t):
                j = i + k
                l = i + t
                m = i + k + t
                yield(i, j, l, m)


def get_interactions(N):
    G2 = []
    G4 = []
    for i,j in two_body_pairs(N):
        G2.append((i,j))

    for i,j,l,m in four_body_pairs(N):
        G4.append((i,j,l,m))
        
    return G2, G4



get_interactions(6)

([(1, 2), (2, 3)], [(1, 3, 2, 4), (1, 4, 2, 5)])



<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 5:</h3>
    <p style="font-size: 16px; color: #333;">
You are now ready to construct the entire circuit and run the counteradiabatic optimization procedure. The final kernel needs to apply Equation 15 for a specified total evolution time $T$ and the `n_steps` number of Trotter steps.  It must also take as input, the indices for the two and four body terms and the thetas to be applied each step, as these cannot be computed within a CUDA-Q kernel.

The helper function `compute_theta` computes the theta parameters for you, using a few additional functions in accordance with the equations defined in the paper.
</div>


In [25]:
import cudaq
import numpy as np
from math import pi

# RZZ gate implementation (needed for both blocks)
@cudaq.kernel
def rzz_gate(q0: cudaq.qubit, q1: cudaq.qubit, theta: float):
    """
    Implements e^(-i*theta*Z⊗Z/2) using:
    RZZ(θ) = CNOT(q0,q1) · RZ(θ,q1) · CNOT(q0,q1)
    """
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)

# Two-qubit rotation block R_YZ(θ) R_ZY(θ)
@cudaq.kernel
def apply_two_body_block(qubits: cudaq.qview, i: int, j: int, theta: float):
    """
    Apply the 2-qubit block from Figure 3 in the paper.
    This implements R_YiZj(θ) R_ZiYj(θ)
    
    Args:
        qubits: quantum register
        i, j: qubit indices (i < j)
        theta: rotation angle (will be 4*θ(nΔt) in the main circuit)
    """
    # First block: R_YZ(theta)
    rx(pi/2, qubits[i])     # Basis change Y -> Z
    h(qubits[j])             # Basis change Z -> X for CNOT
    
    rzz_gate(qubits[i], qubits[j], theta)
    
    rx(-pi/2, qubits[i])    # Restore basis
    h(qubits[j])
    
    # Second block: R_ZY(theta)
    h(qubits[i])             # Basis change Z -> X
    rx(pi/2, qubits[j])      # Basis change Y -> Z
    
    rzz_gate(qubits[i], qubits[j], theta)
    
    h(qubits[i])             # Restore basis
    rx(-pi/2, qubits[j])

# Four-qubit rotation block
@cudaq.kernel
def apply_four_body_block(qubits: cudaq.qview, i: int, i_t: int, i_k: int, i_kt: int, theta: float):
    """
    Apply the 4-qubit block from Figure 4 in the paper.
    This implements the four rotations:
    R_YZZZ, R_ZYZZ, R_ZZYZ, R_ZZZY
    
    Args:
        qubits: quantum register
        i, i_t, i_k, i_kt: four qubit indices for positions i, i+t, i+k, i+k+t
        theta: rotation angle (will be 8*θ(nΔt) in the main circuit)
    """
    indices = [i, i_t, i_k, i_kt]
    
    # The figure shows a complex decomposition with 10 RZZ gates and 28 single-qubit gates
    # For each of the 4 rotations (Y on one qubit, Z on the others):
    
    # Rotation 1: Y on qubit i, Z on others
    rx(pi/2, qubits[i])      # Y basis for qubit i
    h(qubits[i_t])
    h(qubits[i_k])
    h(qubits[i_kt])
    
    # Multi-controlled Z rotation (simplified - actual circuit from paper is more complex)
    # Apply pairwise RZZ gates
    rzz_gate(qubits[i], qubits[i_t], theta/4)
    rzz_gate(qubits[i], qubits[i_k], theta/4)
    rzz_gate(qubits[i], qubits[i_kt], theta/4)
    rzz_gate(qubits[i_t], qubits[i_k], theta/4)
    rzz_gate(qubits[i_t], qubits[i_kt], theta/4)
    rzz_gate(qubits[i_k], qubits[i_kt], theta/4)
    
    h(qubits[i_t])
    h(qubits[i_k])
    h(qubits[i_kt])
    rx(-pi/2, qubits[i])
    
    # Rotation 2: Y on qubit i+t, Z on others
    rx(pi/2, qubits[i_t])
    h(qubits[i])
    h(qubits[i_k])
    h(qubits[i_kt])
    
    rzz_gate(qubits[i_t], qubits[i], theta/4)
    rzz_gate(qubits[i_t], qubits[i_k], theta/4)
    rzz_gate(qubits[i_t], qubits[i_kt], theta/4)
    rzz_gate(qubits[i], qubits[i_k], theta/4)
    rzz_gate(qubits[i], qubits[i_kt], theta/4)
    rzz_gate(qubits[i_k], qubits[i_kt], theta/4)
    
    h(qubits[i])
    h(qubits[i_k])
    h(qubits[i_kt])
    rx(-pi/2, qubits[i_t])
    
    # Rotation 3: Y on qubit i+k, Z on others
    rx(pi/2, qubits[i_k])
    h(qubits[i])
    h(qubits[i_t])
    h(qubits[i_kt])
    
    rzz_gate(qubits[i_k], qubits[i], theta/4)
    rzz_gate(qubits[i_k], qubits[i_t], theta/4)
    rzz_gate(qubits[i_k], qubits[i_kt], theta/4)
    rzz_gate(qubits[i], qubits[i_t], theta/4)
    rzz_gate(qubits[i], qubits[i_kt], theta/4)
    rzz_gate(qubits[i_t], qubits[i_kt], theta/4)
    
    h(qubits[i])
    h(qubits[i_t])
    h(qubits[i_kt])
    rx(-pi/2, qubits[i_k])
    
    # Rotation 4: Y on qubit i+k+t, Z on others
    rx(pi/2, qubits[i_kt])
    h(qubits[i])
    h(qubits[i_t])
    h(qubits[i_k])
    
    rzz_gate(qubits[i_kt], qubits[i], theta/4)
    rzz_gate(qubits[i_kt], qubits[i_t], theta/4)
    rzz_gate(qubits[i_kt], qubits[i_k], theta/4)
    rzz_gate(qubits[i], qubits[i_t], theta/4)
    rzz_gate(qubits[i], qubits[i_k], theta/4)
    rzz_gate(qubits[i_t], qubits[i_k], theta/4)
    
    h(qubits[i])
    h(qubits[i_t])
    h(qubits[i_k])
    rx(-pi/2, qubits[i_kt])

In [27]:
@cudaq.kernel
def dcqo_circuit(N: int, n_steps: int, G2: list[list[int]], G4: list[list[int]], 
                 thetas_2body: list[float], thetas_4body: list[float]):
    """
    Complete DCQO circuit for LABS problem.
    
    Args:
        N: number of qubits (sequence length)
        n_steps: number of Trotter steps
        G2: list of 2-body interaction pairs [(i,j), ...]
        G4: list of 4-body interaction quartets [(i,i+t,i+k,i+k+t), ...]
        thetas_2body: rotation angles for 2-body terms [n_steps x len(G2)]
        thetas_4body: rotation angles for 4-body terms [n_steps x len(G4)]
    """
    # Allocate qubits
    qubits = cudaq.qvector(N)
    
    # Step 1: Prepare initial state |+⟩^⊗N
    # This is the ground state of H_i = Σ h_i^x σ_i^x
    for i in range(N):
        h(qubits[i])  # |0⟩ -> |+⟩
    
    # Step 2: Apply Trotter steps
    for step in range(n_steps):
        
        # Apply all 2-body terms for this Trotter step
        for idx in range(len(G2)):
            i, j = G2[idx]
            # Get theta for this step and interaction
            theta_idx = step * len(G2) + idx
            theta = thetas_2body[theta_idx]
            
            # Apply R_YZ and R_ZY rotations with angle 4*theta
            apply_two_body_block(qubits, i, j, theta)
        
        # Apply all 4-body terms for this Trotter step
        for idx in range(len(G4)):
            i, i_t, i_k, i_kt = G4[idx]
            # Get theta for this step and interaction
            theta_idx = step * len(G4) + idx
            theta = thetas_4body[theta_idx]
            
            # Apply the four rotations with angle 8*theta
            apply_four_body_block(qubits, i, i_t, i_k, i_kt, theta)
    
    # Step 3: Measure all qubits
    mz(qubits)

In [28]:
import auxiliary_files.labs_utils as utils

def compute_theta(N, n_steps, T, G2, G4):
    """
    Compute rotation angles for each Trotter step.
    
    Args:
        N: number of qubits
        n_steps: number of Trotter steps
        T: total evolution time
        G2: 2-body interaction indices
        G4: 4-body interaction indices
        
    Returns:
        thetas_2body: flat list of angles for 2-body terms
        thetas_4body: flat list of angles for 4-body terms
    """
    dt = T / n_steps  # Time step size
    
    thetas_2body = []
    thetas_4body = []
    
    for n in range(n_steps):
        t = (n + 0.5) * dt  # Mid-point of time step
        
        # Compute λ(t) and its derivative
        lambda_t = utils.lambda_schedule(t, T)
        lambda_dot = utils.lambda_derivative(t, T)
        
        # Compute α(t) from Γ₁ and Γ₂
        gamma1 = utils.compute_gamma1(N, G2, G4, lambda_t)
        gamma2 = utils.compute_gamma2(N, G2, G4, lambda_t)
        alpha_t = -gamma1 / gamma2
        
        # θ(t) = Δt × α(t) × λ̇(t)
        theta_t = dt * alpha_t * lambda_dot
        
        # For 2-body terms: angle is 4*θ(t) (but we pass θ and multiply in kernel)
        for _ in G2:
            thetas_2body.append(4 * theta_t)
        
        # For 4-body terms: angle is 8*θ(t)
        for _ in G4:
            thetas_4body.append(8 * theta_t)
    
    return thetas_2body, thetas_4body

In [29]:
# Problem parameters
N = 8  # Sequence length (start small for testing)
n_steps = 10  # Number of Trotter steps
T = 1.0  # Total evolution time
n_shots = 1000  # Number of measurements

# Get interaction indices from Exercise 4
G2, G4 = get_interactions(N)

# Compute rotation angles
thetas_2body, thetas_4body = compute_theta(N, n_steps, T, G2, G4)

# Run the quantum circuit
result = cudaq.sample(dcqo_circuit, N, n_steps, G2, G4, thetas_2body, thetas_4body, 
                      shots_count=n_shots)

# Extract bitstrings and energies
bitstrings = []
energies = []

for bits, count in result.items():
    # Convert bitstring to list
    bit_list = [int(b) for b in bits]
    
    # Compute LABS energy
    energy = labs_energy_list(bit_list)
    
    # Store
    for _ in range(count):
        bitstrings.append(bit_list)
        energies.append(energy)

# Find best solution from quantum sampling
best_idx = np.argmin(energies)
best_quantum = bitstrings[best_idx]
best_energy = energies[best_idx]

print(f"Best quantum solution: {best_quantum}")
print(f"Energy: {best_energy}")

# Use this to seed MTS (Exercise 6)
quantum_population = [best_quantum] * 100  # Replicate best solution
# Or use diverse solutions:
# quantum_population = bitstrings[:100]

AttributeError: module 'auxiliary_files.labs_utils' has no attribute 'lambda_schedule'

## Generating Quantum Enhanced Results

Recall that the point of this lab is to demonstrate the potential benefits of running a quantum subroutine as a preprocessing step for classical optimization of a challenging problem like LABS. you now have all of the tools you need to try this for yourself.

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 6:</h3>
    <p style="font-size: 16px; color: #333;">
Use your CUDA-Q code to prepare an initial population for your memetic search algorithm and see if you can improve the results relative to a random initial population.  If you are running on a CPU, you will need to run smaller problem instances. The code below sets up the problem

</div>


In [ ]:
# TODO - write code here to sample from your CUDA-Q kernel and used the results to seed your MTS population

The results clearly show that a population sampled from CUDA-Q results in an improved distribution and a lower energy final result. This is exactly the goal of quantum enhanced optimization.  To not necessarily solve the problem, but improve the effectiveness of state-of-the-art classical approaches. 

A few major caveats need to be mentioned here. First, We are comparing a quantum generated population to a random sample.  It is quite likely that other classical or quantum heuristics could be used to produce an initial population that might even beat the counteradiabatic method you used, so we cannot make any claims that this is the best. 

Recall that the point of the counteradiabatic approach derived in the paper is that it is more efficient in terms of two-qubit gates relative to QAOA. The benefits of this regime would only truly come into play in a setting (e.g. larger problem instance) where it is too difficult to produce a good initial population with any know classical heuristic, and the counteradiabatic approach is more efficiently run on a QPU compared to alternatives.

We should also note that we are comparing a single sample of each approach.  Maybe the quantum sample got lucky or the randomly generated population was unlucky and a more rigorous comparison would need to repeat the analysis many times to draw any confidently conclusions.  

The authors of the paper discuss all of these considerations, but propose an analysis that is quite interesting related to the scaling of the technique. Rather than run large simulations ourselves, examine their results below. 


<img src="images/quantum_enhanced_optimization_LABS/tabu_search_results.png" width="900">

The authors computed replicate median (median of solving the problem repeated with same setup) time to solutions (excluding time to sample from QPU) for problem sizes $N=27$ to $N=37$. Two interesting conclusions can be drawn from this. First, standard memetic tabu search (MTS) is generally faster than quantum enhanced (QE) MTS.  But there are two promising trends. For larger problems, the QE-MTS experiments occasionally have excellent performance with times to solution much smaller than all of the MTS data points.  These outliers indicate there are certain instances where QE-MTS could provide much faster time-to-solution. 

More importantly, if a line of best fit is calculated using the median of each set of medians, the slope of the QE-MTS line is smaller than the MTS!  This seems to indicate that QE solution of this problem scales $O(1.24^N)$ which is better than the best known classical heuristic ($O(1.34^N)$) and the best known quantum approach (QAOA - $O(1.46^N)$).

For problems of size of $N=47$ or greater, the authors anticipate that QE-MTS could be a promising technique and produce good initial populations that are difficult to obtain classically. 

The study reinforces the potential of hybrid workflows enhanced by quantum data such that a classical routine is still the primary solver, but quantum computers make it much more effective.  Future work can explore improvements to both the quantum and classical sides, such as including GPU accelerated memetic search on the classical side.

## Self-validation To Be Completed for Phase 1

In this section, explain how you verified your results. Did you calculate solutions by hand for small N? Did you create unit tests? Did you cross-reference your Quantum energy values against your Classical MTS results? Did you check known symmetries?